In [ ]:
# import libraries
import pandas as pd
import plotly.graph_objects as go

In [ ]:
# load data
companies = pd.read_excel("Relational Database.xlsx")
species = pd.read_excel("Species.xlsx")

In [ ]:
# preview companies
companies.head()

In [ ]:
# preview species
species.head()

In [ ]:
# split data on production and method
species["production"] = [method.split("_")[1] for method in species["ID_Prod_Meth"]]
species["method"] = [method.split("_")[2] for method in species["ID_Prod_Meth"]]

In [ ]:
# select macroalgae producers
good_ids = [row["ID_Site"] for _, row in species.iterrows() if row["production"] == "Macroalgae"]
mask_companies = companies["ID_Site"].isin(good_ids)
companies = companies.loc[mask_companies, :].reset_index(drop=True)
mask_species = species["ID_Site"].isin(good_ids)
species = species.loc[mask_species, :].reset_index(drop=True)

In [ ]:
# add method info to companies
companies["method"] = ""
for i in range(companies.shape[0]):
    mask_id = species["ID_Site"] == companies.loc[i, "ID_Site"]
    method_current = species.loc[mask_id, "method"].unique()
    if len(method_current) == 1:
        companies.loc[i, "method"] = method_current
    else:
        print(i)

In [ ]:
# discriminate between harvesting and aquaculture
companies["method_cat"] = ""
for i in range(companies.shape[0]):
    method_current = companies.loc[i, "method"]
    if ("Harvesting" in method_current) & ("Aquaculture" in method_current):
        companies.loc[i, "method_cat"] = "Harvesting & Aquaculture"
    elif "Harvesting" in method_current:
        companies.loc[i, "method_cat"] = "Harvesting"
    else:
        companies.loc[i, "method_cat"] = "Aquaculture"

In [ ]:
# plot map of producers
fig=go.Figure()
config = {'displayModeBar': True}

methods = companies["method_cat"].unique()
colors=["#C63074", "#1C9390", "#F9A115"]
for i in range(len(methods)):
    mask_method = companies["method_cat"] == methods[i]
    fig.add_trace(
        go.Scattermap(
            lon = companies.loc[mask_method, "Long"],
            lat = companies.loc[mask_method, "Lat"],
            mode='markers',
            name=methods[i],
            marker=dict(
                size=10,
                color=colors[i],
            ),
            hovertemplate=companies.loc[mask_method, "Owner_name"] + "<extra></extra>"
        )
    )

fig.update_layout(
    hovermode="closest",
    hoverlabel=dict(
        font_size=14,
        font_family="Montserrat",
        font_color="#FDF2EE",
        bordercolor="#FDF2EE",
    ),
    hoverdistance=100,
    map=dict(
        bearing=0,
        center=dict(
            lat=54,
            lon=2
        ),
        zoom=2,
        style="basic"
    ),
    modebar=dict(
        orientation="v",
    ),
    legend=dict(
        orientation="h",
        font_family="Montserrat",
        font_size=13,
        font_color="#113972",
        bgcolor="white",
        entrywidth=180,
        yanchor="top",
        y=0.0,
        xanchor="center",
        x=0.5
    ),
    margin=dict(
        l=30,
        r=30,
        t=0,
        b=20
    ),
    height=500,
    width=750
)

fig.show()

In [ ]:
fig.write_html("producers.html", include_plotlyjs="cdn")

In [ ]:
companies["method"].unique()